In [2]:
import graphlab

In [3]:
# Load a common image analysis dataset

In [18]:
image_train = graphlab.SFrame('image_train_data/')
image_test = graphlab.SFrame('image_test_data/')

### Exploring the image data

In [20]:
graphlab.canvas.set_target('ipynb')

In [21]:
image_train.show()

# Train a classifier on the row image pixels

In [23]:
raw_pixel_model = graphlab.logistic_classifier.create(image_train,target='label',
                                              features=['image_array'])

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

Logistic regression:

--------------------------------------------------------

Number of examples          : 1919

...

| 6         | 12       | 1.000000  | 2.652847     | 0.446066          | 0.418605            |

| 10        | 16       | 1.000000  | 3.676209     | 0.522668          | 0.465116            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

# Make a prediction with the simple model based on raw pixels

In [24]:
image_test[:3]['image'].show()

In [25]:
image_test[:3]['label']

dtype: str
Rows: 3
['cat', 'automobile', 'cat']

In [26]:
raw_pixel_model.predict(image_test[:3])

dtype: str
Rows: 3
['bird', 'cat', 'bird']

# Evaluating raw_pixel_model on test data

In [28]:
raw_pixel_model.evaluate(image_test)

{'accuracy': 0.48, 'auc': 0.7198413750000011, 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 16
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |     bird     |       dog       |  167  |
 |     dog      |       cat       |  251  |
 |     bird     |    automobile   |  145  |
 |  automobile  |    automobile   |  643  |
 |     cat      |       dog       |  275  |
 |     dog      |       dog       |  403  |
 |     dog      |    automobile   |  108  |
 |     bird     |       bird      |  529  |
 |  automobile  |       bird      |  109  |
 |     bird     |       cat       |  159  |
 +--------------+-----------------+-------+
 [16 rows x 3 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns., 'f1_score': 0.4765981605897964, 'log_loss': 1.2113172241422525, 'precision': 0.47

# Can we improve the model using deep features?

In [29]:
len(image_train)

2005

In [32]:
# deep_learning_model = graphlab.load_model('imagenet_model')
# image_train['deep_features'] = deep_learning_model.extract_features(image_train)
image_train.head()

id,image,label,deep_features,image_array
24,Height: 32 Width: 32,bird,"[0.242871761322,1.09545373917, 0.0, ...","[73.0, 77.0, 58.0, 71.0,68.0, 50.0, 77.0, 69.0, ..."
33,Height: 32 Width: 32,cat,"[0.525087952614, 0.0,0.0, 0.0, 0.0, 0.0, ...","[7.0, 5.0, 8.0, 7.0, 5.0,8.0, 5.0, 4.0, 6.0, 7.0, ..."
36,Height: 32 Width: 32,cat,"[0.566015958786, 0.0,0.0, 0.0, 0.0, 0.0, ...","[169.0, 122.0, 65.0,131.0, 108.0, 75.0, ..."
70,Height: 32 Width: 32,dog,"[1.12979578972, 0.0, 0.0,0.778194487095, 0.0, ...","[154.0, 179.0, 152.0,159.0, 183.0, 157.0, ..."
90,Height: 32 Width: 32,bird,"[1.71786928177, 0.0, 0.0,0.0, 0.0, 0.0, ...","[216.0, 195.0, 180.0,201.0, 178.0, 160.0, ..."
97,Height: 32 Width: 32,automobile,"[1.57818555832, 0.0, 0.0,0.0, 0.0, 0.0, ...","[33.0, 44.0, 27.0, 29.0,44.0, 31.0, 32.0, 45.0, ..."
107,Height: 32 Width: 32,dog,"[0.0, 0.0,0.220677852631, 0.0, ...","[97.0, 51.0, 31.0, 104.0,58.0, 38.0, 107.0, 61.0, ..."
121,Height: 32 Width: 32,bird,"[0.0, 0.23753464222, 0.0,0.0, 0.0, 0.0, ...","[93.0, 96.0, 88.0, 102.0,106.0, 97.0, 117.0, ..."
136,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 7.5737862587, 0.0, ...","[35.0, 59.0, 53.0, 36.0,56.0, 56.0, 42.0, 62.0, ..."
138,Height: 32 Width: 32,bird,"[0.658935725689, 0.0,0.0, 0.0, 0.0, 0.0, ...","[205.0, 193.0, 195.0,200.0, 187.0, 193.0, ..."


# Given the deep features let's train a classifier

In [34]:
deep_features_model = graphlab.logistic_classifier.create(image_train, features=['deep_features'], target='label')

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

WARNING: Detected extremely low variance for feature(s) 'deep_features' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.

Logistic regression:

--------------------------------------------------------

...

| 6         | 13       | 0.250000  | 4.180988     | 0.796752          | 0.770833            |

| 10        | 17       | 0.250000  | 5.802785     | 0.871661          | 0.718750            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

# Apply the deep features model to the first few images of test data

In [35]:
image_test[:3].show()

In [36]:
deep_features_model.predict(image_test[0:3])

dtype: str
Rows: 3
['cat', 'automobile', 'cat']

# Compute the test data accuracy of deep features model

In [37]:
deep_features_model.evaluate(image_test)

{'accuracy': 0.7855, 'auc': 0.9405897083333338, 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 16
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |  automobile  |       cat       |   18  |
 |     bird     |       dog       |   58  |
 |     cat      |       bird      |   68  |
 |  automobile  |       dog       |   6   |
 |     cat      |    automobile   |   34  |
 |     dog      |       bird      |   42  |
 |     bird     |       cat       |  128  |
 |     dog      |    automobile   |   17  |
 |     dog      |       dog       |  713  |
 |     cat      |       dog       |  204  |
 +--------------+-----------------+-------+
 [16 rows x 3 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns., 'f1_score': 0.7861287107505106, 'log_loss': 0.5630659633459484, 'precision': 0.